# Things to do with this project


[1]The weighted/unweighted top artists from top 100 lists

[2]Add new column for what the "real" genre of a track is

[3]Throw everything together, manually assign two classes "normie" or "weirdo" then test classifier algorithms

[4]Other fun analysis/visualizations

## [1] Top 5 Artists
Count all the artists by frequency in the 6 top 100 playlists to create an alternate top 5 artists of the last 6 years [DONE]

Also create a weighted list of artists from this dataframe. So a track that was the most listened to that year counts as 1 point and the 80th most played track of that year counts as .20

Worth considering that some tracks have multiple artists listed, so might require a similar strategy to [2] regarding list of lists. 

## [2] Genre Distribution
We also want to have genre breakdowns, but will need to manually correct for imaginary genres

1. In the gigaDF, there will be a column for "Genres", toList this whole column, yielding a list of lists
2. Iterate through the list of lists, for each inner list iterate through and append each 'genre' to a new list
3. implement a counter collection on that list which will yield genres and how often they appear
4. maybe convert that collection to dictionary if thats easier
5. Iterate through gigaDF, for each row, pull out every element of Genres, compare to the counter or dictionary, and assign the value of a new column "RealGenre" to whichever genre named in the "Genres" column that has the most entries

1-4 is already largely implemented near the bottom of the lab, 5 still needs implemented and tested. 

Ideally, this yields a correct supergenre for each category, in practice it might not, if the "parent" genre is absent in the Genres column for an entry it could be misclassified, or if a subgenre appears with greater frequency than its parent genre it will appear as the parent. Either way test it and see how it goes. 

Absolute worst case scenario, manually apply RealGenre. It might even be worth it to manually apply RealGenre and compare the automated and manual results to see how effective the automation is. Reminder: some genres are NaN and empty string, for some reason

## [3] Machine Learning

Some jams are widely liked, some jams are a bit more specific. We want to classify a song as either a "Normie" song or a "Weirdo" song. This will be done manually. 

Each track has certain attributes that might be used to train classification models:
- Popularity
- Danceability
- Energy
- Loudness
- Speechiness
- Acousticness
- Instrumentalness
- Liveness
- Valence
- Temp

Popularity, as a guess, might be too heavily weighted as a factor. If that happens we will try without it. So, the dimensionality here is 10 or maybe 9. This is significantly less than a trillion (See: Training Deep Learning-based recommender models of 100 trillion parameters over Google Cloud), but might still be fun




Manually assign each track from the top 100s lists a class of either Normie of Weirdo
Train (10f CV) different classification algorithms (below) on the data and compare the effectiveness [Accuracy vs Precision vs Sensitivity vs Specificity]
- kNN
- SVM
- Logistic Regression
- Decision Tree
- Random Forest
- Naive Bayes

# NOTES HERE
- Survivor = a track that has been present in more than one top 100 playlist
- showVisuals is a flag that can be set to True to display the visualizations involved
- One of my top 100s has 101 songs, no idea why spotify did that 

In [31]:
showVisuals = False

from matplotlib import pyplot
import seaborn
import pandas as pd
from collections import defaultdict, OrderedDict
#import collections
from scipy.stats import pareto, gamma
from datetime import date
#from collections import Counter, OrderedDict


megamix = 'liked_songs.csv'
mix2024 = '2024.csv'
mix2023 = '2023.csv'
mix2022 = '2022.csv'
mix2021 = '2021.csv'
mix2020 = '2020.csv'
mix2019 = '2019.csv'
mix1    = 'my_mix.csv'
mix2    = 'my_mix_part_2.csv'



# read the data
likedData = pd.read_csv(megamix)
mixOne  = pd.read_csv(mix1)
mixTwo  = pd.read_csv(mix2)
Data24  = pd.read_csv(mix2024)
Data23  = pd.read_csv(mix2023)
Data22  = pd.read_csv(mix2022)
Data21  = pd.read_csv(mix2021)
Data20  = pd.read_csv(mix2020)
Data19  = pd.read_csv(mix2019)

In [32]:
# print("total songs:", data.shape[0])
likedData.head(5)

,Track ID,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Added By,Added At,Genres,...,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature
0,1pnDvUuAEd6z8bKEsbAjk1,We Don't Trust You,WE DON'T TRUST YOU,"Future,Metro Boomin",2024-03-22,226213,70,NaN,2024-11-30T08:45:27Z,"atl hip hop,hip hop,rap,southern hip hop,trap",...,6,-7.169,0,0.2710,0.478000,0.000439,0.2430,0.346,124.091,4
1,4I5QRJ0vpMpnzR23Lpm3k9,Whiplash,Whiplash,Architects,2024-11-19,226166,65,NaN,2024-11-20T05:04:38Z,"mathcore,melodic metalcore,metalcore,progressi...",...,8,-3.189,0,0.1530,0.000061,0.002570,0.9000,0.423,95.041,4
2,5ZUIPLoTLJZrPQh2kFZEUM,Addicted,Addicted,"Zerb,The Chainsmokers,Ink",2024-03-29,152250,81,NaN,2024-11-09T07:07:53Z,"pop dance,pop",...,5,-7.132,0,0.0374,0.024500,0.000163,0.1110,0.190,120.064,4
3,6BzU6BMqMd4qelkWTjLwKy,Only Children,Reunions,Jason Isbell and the 400 Unit,2020-05-15,237126,46,NaN,2024-11-06T03:00:13Z,"alternative country,folk,new americana,roots rock",...,4,-10.715,0,0.0346,0.655000,0.001790,0.0858,0.236,89.499,4
4,3DnsQ5O4uJw3MlYgvBJSCq,Maxed Out,Maxed Out,Bayker Blankenship,2024-04-19,172607,73,NaN,2024-11-06T02:42:29Z,modern country pop,...,7,-10.778,0,0.0360,0.359000,0.000008,0.1060,0.364,145.915,4


In [33]:
print(likedData.columns.tolist())

['Track ID', 'Track Name', 'Album Name', 'Artist Name(s)', 'Release Date', 'Duration (ms)', 'Popularity', 'Added By', 'Added At', 'Genres', 'Record Label', 'Danceability', 'Energy', 'Key', 'Loudness', 'Mode', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo', 'Time Signature']


In [34]:
# likedData
# Data24 
# Data23
# Data22  
# Data21 
# Data20 
# Data19
likedData[["Track Name", "Artist Name(s)"]]

,Track Name,Artist Name(s)
0,We Don't Trust You,"Future,Metro Boomin"
1,Whiplash,Architects
2,Addicted,"Zerb,The Chainsmokers,Ink"
3,Only Children,Jason Isbell and the 400 Unit
4,Maxed Out,Bayker Blankenship
...,...,...
732,Gotta Get It,"Ruin Your Day,D.I. Da Hennyman,Spiff"
733,In the Pines (Oakland),Fantastic Negrito
734,OooWee,"Rapsody,Anderson .Paak"
735,Gillette,"199X,DrDisrespect"


In [35]:
# Data24 
# Data23
# Data22  
# Data21 
# Data20 
# Data19

#Find songs that last through the years
#OmniTracksDF = Data24[["Track Name", "Artist Name(s)"]]
years = ["2024", "2023", "2022", "2021", "2020", "2019"]
list_of_lists = []
for i in years:
    #print(i)
    tracks = []
    artists = []
    # we dont need year as a list, just at the end, for all rows where year is NaN, set year to i
    # unless for some stupid reason python thinks a list is more efficient, for symmetry we'll do it this way for the moment
    year = []

    #This feels cartoonishly overcomplicated

    
    
    if i == "2024":
        col_one_list = Data24["Track Name"].tolist()
        col_two_list = Data24["Artist Name(s)"].tolist()

        #list_of_lists = []
        rowsX = 100
        colsX = 3

        for i in range(rowsX):
            inner_list = []
            inner_list.append(col_one_list[i])
            inner_list.append(col_two_list[i])
            inner_list.append("2024")
            list_of_lists.append(inner_list)
    if i == "2023":
        col_one_list = Data23["Track Name"].tolist()
        col_two_list = Data23["Artist Name(s)"].tolist()

        #list_of_lists = []
        rowsX = 100
        colsX = 3

        for i in range(rowsX):
            inner_list = []
            inner_list.append(col_one_list[i])
            inner_list.append(col_two_list[i])
            inner_list.append("2023")
            list_of_lists.append(inner_list)
    if i == "2022":
        col_one_list = Data22["Track Name"].tolist()
        col_two_list = Data22["Artist Name(s)"].tolist()

        #list_of_lists = []
        rowsX = 100
        colsX = 3

        for i in range(rowsX):
            inner_list = []
            inner_list.append(col_one_list[i])
            inner_list.append(col_two_list[i])
            inner_list.append("2022")
            list_of_lists.append(inner_list)
    if i == "2021":
        col_one_list = Data21["Track Name"].tolist()
        col_two_list = Data21["Artist Name(s)"].tolist()

        #list_of_lists = []
        rowsX = 100
        colsX = 3

        for i in range(rowsX):
            inner_list = []
            inner_list.append(col_one_list[i])
            inner_list.append(col_two_list[i])
            inner_list.append("2021")
            list_of_lists.append(inner_list)
    if i == "2020":
        col_one_list = Data20["Track Name"].tolist()
        col_two_list = Data20["Artist Name(s)"].tolist()

        #list_of_lists = []
        rowsX = 100
        colsX = 3

        for i in range(rowsX):
            inner_list = []
            inner_list.append(col_one_list[i])
            inner_list.append(col_two_list[i])
            inner_list.append("2020")
            list_of_lists.append(inner_list)
    if i == "2019":
        col_one_list = Data19["Track Name"].tolist()
        col_two_list = Data19["Artist Name(s)"].tolist()

        #list_of_lists = []
        rowsX = 100
        colsX = 3

        for i in range(rowsX):
            inner_list = []
            inner_list.append(col_one_list[i])
            inner_list.append(col_two_list[i])
            inner_list.append("2019")
            list_of_lists.append(inner_list)

dfX = pd.DataFrame(list_of_lists)
dfX = dfX.rename(columns={0: "Track", 1: "Artist(s)", 2: "Year"})
dfX.shape
dfX.head(5)
#dfX = pd.DataFrame()
#dfX["Track"] = Data24["Track Name"]
#dfX["Artist(s)"] = Data24["Artist Name(s)"]
#dfX.head()

#data = []
#for row in some_function_that_yields_data():
#    data.append(row)

#df = pd.DataFrame(data)

,Track,Artist(s),Year
0,Dante,Northlane,2024
1,RATATATA,"BABYMETAL,Electric Callboy",2024
2,labour,Paris Paloma,2024
3,Rucón,Alemán,2024
4,Even Flow,Pearl Jam,2024


In [36]:
# I think we want keep=True here because we want to include the first year that the survivor appears
dfX[dfX.duplicated('Track', keep=False) == True]

,Track,Artist(s),Year
27,Love Chains,BOYY,2024
34,Lone Digger,Caravan Palace,2024
61,Down,"Hemlock Ernst,Kenny Segal",2024
66,Come Check This,FETISH,2024
77,N.Y. State of Mind,Nas,2024
...,...,...,...
579,Bodysnatchers,Dave Rawlings Machine,2019
585,WISH YOU WELL,Amir Obe,2019
588,Out Of Control,Hoobastank,2019
597,Caterpillar (feat. Eminem & King Green),"Royce Da 5'9,Eminem,King Green",2019


In [37]:
dfX1 = dfX[dfX.duplicated('Track', keep=False) == True].copy()
dfX1["Years of Apperance"] = dfX1.groupby('Track', as_index=False)['Track'].cumcount()
#dfX1.head
dfX1 = dfX1.sort_values("Track")


#dfX1 = dfX1.sort_values("Years of Apperance")
#print(dfX1.to_string())

#But since the immediate goal is to find songs that showed up the most

In [38]:
#So 25 songs have appeared in 3 years, 1 appeared in 4 years, and sadly none have run the table

In [39]:
#Okay so this isn't actually that cool because the split is like, 3 is the most for billy joel (2024), so we should look at all top 100s for this

# count songs per artist
artists = defaultdict(int)
for i,song in Data24.iterrows():
    if isinstance(song['Artist Name(s)'], str):
    	for musician in song['Artist Name(s)'].split(','):
    		artists[musician] += 1

# sort for chart
artists = pd.DataFrame(artists.items(), columns=['Artist', 'Num Songs']
                          ).sort_values('Num Songs', ascending=False).reset_index(drop=True)
print("number of unique artists:", artists.shape[0])


if(showVisuals):
    pyplot.figure(figsize=(18, 6))
    pyplot.bar(artists['Artist'], artists['Num Songs'])
    pyplot.xticks(visible=False)
    pyplot.xlabel(artists.columns[0])
    pyplot.ylabel(artists.columns[1])
    pyplot.title('everybody')
    pyplot.show()

number of unique artists: 117


In [40]:
if(showVisuals):
    pyplot.figure(figsize=(18, 10))
    pyplot.bar(artists['Artist'][:50], artists['Num Songs'][:50])
    pyplot.xticks(rotation=80)
    pyplot.xlabel(artists.columns[0])
    pyplot.ylabel(artists.columns[1])
    pyplot.title('top 50');

In [41]:
tempGigaDF1 = pd.concat([Data24, Data23], ignore_index=True)
tempGigaDF2 = pd.concat([Data22, Data21], ignore_index=True)
tempGigaDF3 = pd.concat([Data20, Data19], ignore_index=True)

tempGigaDF = pd.concat([tempGigaDF1, tempGigaDF2], ignore_index=True)
tempGigaDF = pd.concat([tempGigaDF, tempGigaDF3], ignore_index=True)

# count songs per genre

# < so this section actually does the thing we wanted to do using Counter
# my gut says it would be quicker to toList the genres column into a list of lists instead of iterrating through the entire dataframe row by row

genres = defaultdict(int)
for i,song in tempGigaDF.iterrows():
    if type(song['Genres']) is str: # some times there aren't any, and this is NaN
        for genre in song['Genres'].split(','):
            if len(genre) > 0: # empty string seems to be a legit genre
                genres[genre] += 1

# sort for chart
genres = pd.DataFrame(genres.items(), columns=['Genre', 'Num Songs']
                          ).sort_values('Num Songs', ascending=False).reset_index(drop=True)


# >







print("number of unique genres:", genres.shape[0])


if(showVisuals):
    pyplot.figure(figsize=(18, 6))
    pyplot.bar(genres['Genre'], genres['Num Songs'])
    pyplot.xticks(visible=False)
    pyplot.xlabel(genres.columns[0])
    pyplot.ylabel(genres.columns[1])
    pyplot.title('All the genera');

number of unique genres: 563


In [42]:
if(showVisuals):
    pyplot.figure(figsize=(18, 10))
    pyplot.bar(genres['Genre'][:50], genres['Num Songs'][:50])
    pyplot.xticks(rotation=80)
    pyplot.xlabel(genres.columns[0])
    pyplot.ylabel(genres.columns[1])
    pyplot.title('top 50');

#pyplot.savefig("top 50.png", bbox_inches="tight")

In [43]:
if(showVisuals):
    pyplot.figure(figsize=(18, 1))
    pyplot.bar(genres['Genre'][-50:], genres['Num Songs'][-50:])
    pyplot.xticks(rotation=80)
    pyplot.xlabel(genres.columns[0])
    pyplot.ylabel(genres.columns[1])
    pyplot.title('bottom 50');

In [44]:
genres.head

<bound method NDFrame.head of                      Genre  Num Songs
0                  hip hop         58
1        alternative metal         43
2                      rap         39
3            new americana         30
4             gangster rap         30
..                     ...        ...
558        indie poptimism          1
559          bronx hip hop          1
560                bagpipe          1
561  bagpipe marching band          1
562               etherpop          1

[563 rows x 2 columns]>

In [45]:
genres.loc[genres['Num Songs'] == 1]

,Genre,Num Songs
333,garage punk blues,1
334,one-person band,1
335,halifax indie,1
336,dark folk,1
337,dark trap,1
...,...,...
558,indie poptimism,1
559,bronx hip hop,1
560,bagpipe,1
561,bagpipe marching band,1


In [46]:
#Okay so, 563 genres in the top 600.....which looks weird to me, maybe check that
#230 genres are singles
#so 333 genres are at least 2, which....OH you know what I bet this is because a track can have many genres
#Yeah okay it definitely is

In [47]:
display(genres)

,Genre,Num Songs
0,hip hop,58
1,alternative metal,43
2,rap,39
3,new americana,30
4,gangster rap,30
...,...,...
558,indie poptimism,1
559,bronx hip hop,1
560,bagpipe,1
561,bagpipe marching band,1


In [48]:
%%time
#Using the %%time function, this method does appear to be quicker than iterrating through all the rows of the DF. Neat. 

tempList = tempGigaDF['Genres'].tolist()
#print(len(tempList)) # = 601, which might be why the initial method checks for NaN and empty string

tempList2 = []
for item in tempList:
    #print(item)
    if type(item) is str:
        for genre in item.split(','):
            #print(genre)
            tempList2.append(genre)
#print(len(tempList2))


counter = collections.Counter(tempList2)
#print(len(counter)) # = 563
tempDict = OrderedDict(counter.most_common())

tempDF = pd.DataFrame.from_dict(tempDict, orient='index').reset_index()
display(tempDF)

,index,0
0,hip hop,58
1,alternative metal,43
2,rap,39
3,new americana,30
4,gangster rap,30
...,...,...
558,canadian singer-songwriter,1
559,orgcore,1
560,polish folk,1
561,futuristic swag,1


CPU times: total: 15.6 ms
Wall time: 6.98 ms


In [49]:
%%time
genres = defaultdict(int)
for i,song in tempGigaDF.iterrows():
    if type(song['Genres']) is str: # some times there aren't any, and this is NaN
        for genre in song['Genres'].split(','):
            if len(genre) > 0: # empty string seems to be a legit genre
                genres[genre] += 1

# sort for chart
genres = pd.DataFrame(genres.items(), columns=['Genre', 'Num Songs']
                          ).sort_values('Num Songs', ascending=False).reset_index(drop=True)


display(genres)

,Genre,Num Songs
0,hip hop,58
1,alternative metal,43
2,rap,39
3,new americana,30
4,gangster rap,30
...,...,...
558,indie poptimism,1
559,bronx hip hop,1
560,bagpipe,1
561,bagpipe marching band,1


CPU times: total: 31.2 ms
Wall time: 31.5 ms


In [50]:
#We can actually use the cell above to get an artist count from the 6 top 100s list

tempList = tempGigaDF['Artist Name(s)'].tolist()
tempList2 = []
for item in tempList:
    if type(item) is str:
        for artist in item.split(','):
            tempList2.append(artist)
counter = collections.Counter(tempList2)
tempDict = OrderedDict(counter.most_common())
tempDF = pd.DataFrame.from_dict(tempDict, orient='index').reset_index()
tempDF.head(20)

#Okay, ignoring OurVinyl, even though thats an interesting an sensible result, our top 5 artists from 2019-2024 by appearances in a top 100 list are: 
# Snow Tha Product, Colter Wall, Eminem, Nate Dogg, Flatbush Zombies

,index,0
0,Snow Tha Product,11
1,Colter Wall,9
2,OurVinyl,8
3,Eminem,8
4,Nate Dogg,8
5,Flatbush Zombies,5
6,John Moreland,5
7,Aesop Rock,5
8,Tyler Childers,4
9,Mastodon,4


In [51]:
#display(tempGigaDF)
#top600DF = tempGigaDF[['Track Name', 'Artist Name(s)']].copy()
#display(dfX)
tempDFx = dfX.copy()
tempDFx["Rank"] = (tempDFx.index)%100 + 1
tempDFx["Change"] = "-"
#display(tempDFx)

duplicatesDF = tempDFx[tempDFx.duplicated('Track', keep=False) == True].copy()
duplicatesDF = duplicatesDF.sort_values(['Track', 'Year'], ascending=[True, True])
duplicatesDF = duplicatesDF.reset_index()

tempTrackStore = ''
tempRank = 0
tempYear = 0
for index, row in duplicatesDF.iterrows():
    thisTrack = row['Track']
    thisRank = row['Rank']
    thisYear = int(row['Year'])
    thisIndex = row['index']
    if (thisTrack == tempTrackStore) and (tempYear==thisYear-1):
        #print(thisTrack)
        #print("Now: ", thisRank, " vs Previous: ", tempRank)
        #duplicatesDF.at[index, 'Change'] = str(tempRank-thisRank)
        tempDFx.at[thisIndex, 'Change'] = str(tempRank-thisRank)
    tempTrackStore = thisTrack
    tempRank = thisRank
    tempYear = thisYear
        
    
#display(myDF)
#display(duplicatesDF.sort_values(['Track', 'Year'], ascending=False))
#dfX[dfX.duplicated('Track', keep=False) == True].copy()

#with pd.option_context("display.max_rows", 1000):
display(tempDFx)

,Track,Artist(s),Year,Rank,Change
0,Dante,Northlane,2024,1,-
1,RATATATA,"BABYMETAL,Electric Callboy",2024,2,-
2,labour,Paris Paloma,2024,3,-
3,Rucón,Alemán,2024,4,-
4,Even Flow,Pearl Jam,2024,5,-
...,...,...,...,...,...
595,Good Man,Raphael Saadiq,2019,96,-
596,I Miss You,blink-182,2019,97,-
597,Caterpillar (feat. Eminem & King Green),"Royce Da 5'9,Eminem,King Green",2019,98,-
598,Los Ageless,St. Vincent,2019,99,-


In [52]:
#tempDFx.to_csv('top600.csv', index=False) 

In [56]:
pdList = [likedData, mixOne, mixTwo, Data24, Data23, Data22, Data21, Data20, Data19]  
maxDF = pd.concat(pdList) 
# 1691 pre drop
# Due to column "Added At" duplicates dont exist, need to drop that column first
maxDF.drop('Added At', axis=1, inplace=True)
maxDF = maxDF.drop_duplicates() # 1535 after drop, 156 entries removed
#display(maxDF)
#maxDF.to_csv('giantList.csv', index=False) 